<h1 style="font-family:Impact,Arial;font-size:30px;">37005 Fundamental of Derivative Security Pricing - Spring 2024</h1>
<h1 style="font-family:Impact,Arial;font-size:20px;">Group Assignment Part 1</h1>
<p style="font-family:Arial;">Group 1</p>
<ul>
<li>Hayoung Lee</li>
<li>Quoc Thai Tran</li>
<li>Alexis Cullet</li>
<li>Ziqi Zhou</li>
</ul>


# Import package


In [12]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, fsolve, basinhopping
from scipy.stats import norm

# Import data


In [3]:
def data_preprocessing():
    df = pd.read_excel('spx_quotedata20220308_all.xlsx')
    df.drop(columns=['Last Sale', 'Calls', 'Net', 'IV', 'Delta', 'Gamma', 'Open Interest', 'Puts', 'Last Sale.1', 'Net.1', 'IV.1', 'Delta.1', 'Gamma.1', 'Open Interest.1'], inplace=True)
    df.rename(columns={'Bid': 'Call Bid', 'Ask': 'Call Ask', 'Volume': 'Call Volume', 'Bid.1':'Put Bid', 'Ask.1':'Put Ask', 'Volume.1':'Put Volume'}, inplace=True)
    df['Expiration Date']=pd.to_datetime(df['Expiration Date'], format='%a %b %d %Y')
    df['Call Price']=(df['Call Bid']+df['Call Ask'])/2
    df['Put Price']=(df['Put Bid']+df['Put Ask'])/2
    df.sort_values(by=['Expiration Date', 'Strike'], ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    columns = df.columns.tolist()
    return df[columns[:2]+['Call Price']+columns[2:6]+['Put Price']+columns[6:-2]]
data_preprocessing()

,Expiration Date,Call Bid,Call Price,Call Ask,Call Volume,Strike,Put Bid,Put Price,Put Ask,Put Volume
0,2022-03-09,1151.7,1160.30,1168.9,0,3000,0.0,0.025,0.05,6
1,2022-03-09,951.7,960.40,969.1,0,3200,0.0,0.025,0.05,4
2,2022-03-09,851.2,860.25,869.3,0,3300,0.0,0.025,0.05,2182
3,2022-03-09,751.7,760.40,769.1,0,3400,0.0,0.025,0.05,360
4,2022-03-09,651.2,660.30,669.4,0,3500,0.0,0.025,0.05,1264
...,...,...,...,...,...,...,...,...,...,...
5886,2026-12-18,34.3,74.30,114.3,0,7200,2602.4,2698.400,2794.40,80
5887,2026-12-18,0.0,150.00,300.0,0,8000,3299.5,3395.500,3491.50,3
5888,2026-12-18,0.0,150.00,300.0,0,8600,3832.4,3928.400,4024.40,5
5889,2026-12-18,0.0,150.00,300.0,0,9000,4190.1,4286.100,4382.10,2


# Task 1

In [18]:
def task_1(stock):
    df = data_preprocessing()
    maturity = list(set(df['Expiration Date']))
    dividend = []
    discount_factor = []
    for date in maturity:
        data = df[df['Expiration Date']==date]
        strike = data['Strike'].to_numpy()
        call = data['Call Price'].to_numpy()
        put = data['Put Price'].to_numpy()
        def target_minimizer(x):
            calc = x[0]*stock-x[1]*strike-(call-put)
            return np.sum(np.power(calc-np.mean(calc),2))
        result = minimize(target_minimizer, np.ones(2), constraints={
            'type':'eq',
            'fun':lambda x: np.sum(x[0]*stock-x[1]*strike-(call-put))
        }).x
        dividend.append(result[0])
        discount_factor.append(result[1])
    information = pd.DataFrame({'Maturity':pd.to_datetime(maturity), 'Dividend Discount Factor':dividend, 'Zero Coupon Bond':discount_factor})
    information.sort_values(by=['Maturity'], ascending=True, inplace=True)
    information.reset_index(drop=True, inplace=True)
    return information
print("Task 1 result")
task_1(4170.7002)

Task 1 result


,Maturity,Dividend Discount Factor,Zero Coupon Bond
0,2022-03-09,0.997496,0.999965
1,2022-03-11,0.997462,1.000079
2,2022-03-14,0.997263,1.000150
3,2022-03-16,0.997030,0.999847
4,2022-03-18,0.996945,0.999927
5,2022-03-21,0.997252,1.000222
6,2022-03-23,0.997327,1.000346
7,2022-03-25,0.996831,0.999867
8,2022-03-28,0.996913,0.999969
9,2022-03-30,0.996952,1.000063


# Task 2

$$
C(0)=S(0)D(0,T_i)\phi(d_1)-KB(0,T_i)\phi(d_2)
$$
$$
d_{1,2}=\frac{Log(\frac{S(0)}{K})-Log(\frac{B(0,T_i)}{D(0,T_i)})\pm\frac{1}{2}\sum_{i=1}^{n}\sigma _i^2(T_i-T_{i-1})}{\sqrt{\sum_{i=1}^{n}\sigma _i^2(T_i-T_{i-1})}}
$$

In [20]:
def task_2(stock):
    data = data_preprocessing()
    dfs = task_1(stock)
    today = pd.to_datetime('2022-03-08')
    sigma = [[],[]]
    prev_sums = [0,0]
    for i in range(len(dfs)):
        ext_data = data[data['Expiration Date']==dfs.iloc[i,0]].reset_index(drop=True)
        ext_data['diff']=np.abs(ext_data['Strike']-stock*dfs.iloc[i,1]/dfs.iloc[i,2])
        chosen_data = list(ext_data.loc[ext_data['diff'].idxmin()])[:-1]
        delta_t = (chosen_data[0]-today).days/((today+pd.DateOffset(years=1))-today).days if i==0 else (chosen_data[0]-dfs.iloc[i-1,0]).days/((dfs.iloc[i-1,0]+pd.DateOffset(years=1))-dfs.iloc[i-1,0]).days
        def solver(x, type):
            prev_sum = prev_sums[0] if type=='bid' else prev_sums[1]
            vol = (prev_sum+(x**2)*delta_t)
            d1 = (np.log(stock/chosen_data[5])-np.log(dfs.iloc[i,2]/dfs.iloc[i,1])+1/2 * vol)/(np.sqrt(vol))
            d2 = d1-np.sqrt(vol)
            return stock*dfs.iloc[i,1]*norm.cdf(d1)-chosen_data[5]*dfs.iloc[i,2]*norm.cdf(d2) - chosen_data[1 if type=='bid' else 3]
        sigma[0].append(fsolve(solver, 0.5, args=('bid'))[0])
        sigma[1].append(fsolve(solver, 0.5, args=('ask'))[0])
        prev_sums[0] += (sigma[0][-1]**2)*delta_t
        prev_sums[1] += (sigma[1][-1]**2)*delta_t
    return pd.DataFrame({'Maturity':dfs['Maturity'], 'Call Implied Volatility':sigma[0], 'Put Implied Volatility':sigma[1]})
task_2(4170.7002)

C:\Users\1tran\AppData\Local\Temp\ipykernel_11848\169387746.py:18: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  sigma[0].append(fsolve(solver, 0.5, args=('bid'))[0])
C:\Users\1tran\AppData\Local\Temp\ipykernel_11848\169387746.py:18: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  sigma[0].append(fsolve(solver, 0.5, args=('bid'))[0])
C:\Users\1tran\AppData\Local\Temp\ipykernel_11848\169387746.py:18: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  sigma[0].append(fsolve(solver, 0.5, args=('bid'))[0])
C:\Users\1tran\AppData\Local\Temp\ipykernel_11848\169387746.py:19: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  sigma[1].append(fsolve(solver, 0.5, args=('ask'))[0])
C:\Us

,Maturity,Call Implied Volatility,Put Implied Volatility
0,2022-03-09,0.341898,0.413275
1,2022-03-11,0.359346,0.348856
2,2022-03-14,0.253594,0.237777
3,2022-03-16,0.399911,0.402351
4,2022-03-18,0.278176,0.325393
5,2022-03-21,0.277750,0.245578
6,2022-03-23,0.333024,0.332520
7,2022-03-25,0.318958,0.322632
8,2022-03-28,0.207789,0.211183
9,2022-03-30,0.315007,0.316442


# Task 4